# Eager execution

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

tf.enable_eager_execution()

## Are we executing eagerly?

In [ ]:
tfe = tf.contrib.eager
tf.executing_eagerly()        # => True

## compute some product

In [ ]:
x = tf.constant([[2]])
m = tf.matmul(x, x)
print("hello, {}".format(m))  # => "hello, [[4.]]"

### Look at shapes of types of Tensors

 x is a Tensor wih shape 1, 1

In [ ]:
x

In [ ]:
tf.constant([[2, 3, 4]])

In [ ]:
tf.constant([[2, 3, 4], [1, 2, 3]])

In [ ]:
tf.constant([[[2, 2], [3, 2], [3 ,4]], [[1,5.0], [2, 1], [3, 3]]])

## Import data

In [ ]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images,
                                              mnist.train.labels))

### Example of linear model learning

In [ ]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.W = tfe.Variable(5., name='weight')
        self.B = tfe.Variable(10., name='bias')
    def predict(self, inputs):
        return inputs * self.W + self.B

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 2000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# The loss function to be optimized
def loss(model, inputs, targets):
    error = model.predict(inputs) - targets
    return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, [model.W, model.B])

# Define:
# 1. A model.
# 2. Derivatives of a loss function with respect to model parameters.
# 3. A strategy for updating the variables based on the derivatives.
model = Model()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

# Training loop
for i in range(300):
    grads = grad(model, training_inputs, training_outputs)
    optimizer.apply_gradients(zip(grads, [model.W, model.B]),
                              global_step=tf.train.get_or_create_global_step())
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))